## Case Pyspark - Jump Start - Turma 04 - 
### Marcos Gonçalves

<center>
    
![PysparkxPandas](PysparkxPandas.png)

</center>

## Utilizado uma questão do Case Pandas e feito uma comparação da performance com o Pyspark 

## Importando Bibliotecas

In [1]:
import pandas as pd
import time
from pyspark.sql.functions import col, lit, regexp_replace, to_date, split, explode, floor, year, row_number
from pyspark.sql.window import Window

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Spark').getOrCreate()
spark

## Medir Performance

In [3]:
# Pandas
pandas_start = time.time()
pandas_end = time.time()
pandas_time = pandas_end - pandas_start

#Pyspark
pyspark_start = time.time()
pyspark_end = time.time()
pyspark_time = pyspark_end - pyspark_start

## Carregando os Arquivos

### Utilizando Pyspark

In [5]:
parametros = {
    'delimiter' : '\t',
    'header': True,
    'inferSchema': True
}

In [14]:
#title_basics
pyspark_start = time.time()

path_titlebasics = r'C:\Users\marcosgoncalves\Desktop\Dados_Programacao\JumpStart2024\Python\Case_Python\title.basics.tsv'
df_pyspark_title_basics = spark.read.options(**parametros).csv(path_titlebasics)

pyspark_end = time.time()
print(f'Tempo de execução com Pyspark: {pyspark_end - pyspark_start:.2f} segundos')

Tempo de execução com Pyspark: 32.56 segundos


In [15]:
#title_ratings
pyspark_start = time.time()

path_titleratings = r'C:\Users\marcosgoncalves\Desktop\Dados_Programacao\JumpStart2024\Python\Case_Python\title.ratings.tsv'
df_pyspark_title_ratings = spark.read.options(**parametros).csv(path_titleratings)

pyspark_end = time.time()
print(f'Tempo de execução com Pyspark: {pyspark_end - pyspark_start:.2f} segundos')

Tempo de execução com Pyspark: 4.03 segundos


In [16]:
#title_akas
pyspark_start = time.time()

path_titleakas = r'C:\Users\marcosgoncalves\Desktop\Dados_Programacao\JumpStart2024\Python\Case_Python\title.akas.tsv'
df_pyspark_title_akas = spark.read.options(**parametros).csv(path_titleakas)

pyspark_end = time.time()
print(f'Tempo de execução com Pyspark: {pyspark_end - pyspark_start:.2f} segundos')

Tempo de execução com Pyspark: 146.56 segundos


### Utilizando Pandas

In [17]:
#title_basics
pandas_start = time.time()

chunk_size = 50000
chunkstb = []

try:
    for chunk in pd.read_csv(r'C:\Users\marcosgoncalves\Desktop\Dados_Programacao\JumpStart2024\Python\Case_Python\title.basics.tsv', sep= '\t', chunksize=chunk_size):
        chunkstb.append(chunk)
    df_pandas_title_basics = pd.concat(chunkstb, axis=0)
except pd.errors.ParserError as e:
    print(f'Erro ao ler o arquivo')

pandas_end = time.time()
print(f'Tempo de execução com Pandas: {pandas_end - pandas_start:.2f} segundos')

Tempo de execução com Pandas: 346.37 segundos


In [18]:
#title_ratings
pandas_start = time.time()

df_pandas_title_ratings = pd.read_csv(r'C:\Users\marcosgoncalves\Desktop\Dados_Programacao\JumpStart2024\Python\Case_Python\title.ratings.tsv', sep= '\t')

pandas_end = time.time()
print(f'Tempo de execução com Pandas: {pandas_end - pandas_start:.2f} segundos')

Tempo de execução com Pandas: 12.53 segundos


In [2]:
#title_akas
pandas_start = time.time()

chunk_size = 50000
chunksta = []

try:
    for chunk in pd.read_csv(r'C:\Users\marcosgoncalves\Desktop\Dados_Programacao\JumpStart2024\Python\Case_Python\title.akas.tsv', sep= '\t', chunksize=chunk_size):
        chunksta.append(chunk)
    df_pandas_title_akas = pd.concat(chunksta, axis=0)
except pd.errors.ParserError as e:
    print(f'Erro ao ler o arquivo')

pandas_end = time.time()
print(f'Tempo de execução com Pandas: {pandas_end - pandas_start:.2f} segundos')

Tempo de execução com Pandas: 1535.36 segundos


### Aplicando um filtro para trabalhar apenas com filmes identificados por movie, tvMovie, short e tvShort, descartando as demais categorias que não se tratam de filmes

#### Reduzindo a base para 1.848.568 linhas e mantendo as 9 colunas

#### Pandas

In [20]:
df_pandas_title_basics.shape

(10918941, 9)

In [21]:
pandas_start = time.time()

df_pandas_title_basics = df_pandas_title_basics[(df_pandas_title_basics['titleType'] == 'movie') | (df_pandas_title_basics['titleType'] =='tvMovie') 
|(df_pandas_title_basics['titleType'] =='short') | (df_pandas_title_basics['titleType'] =='tvShort')]

pandas_end = time.time()
print(f'Tempo de execução com Pandas: {pandas_end - pandas_start:.2f} segundos')

Tempo de execução com Pandas: 466.64 segundos


In [22]:
df_pandas_title_basics.shape

(1848568, 9)

#### Pyskpark

In [23]:
df_pyspark_title_basics.count()

10918941

In [24]:
pyspark_start = time.time()

df_pyspark_title_basics = df_pyspark_title_basics.filter(
    (col('titleType') == 'movie') |
    (col('titleType') == 'tvMovie') |
    (col('titleType') == 'short') |
    (col('titleType') == 'tvShort')
)

pyspark_end = time.time()
print(f'Tempo de execução com Pyspark: {pyspark_end - pyspark_start:.2f} segundos')

Tempo de execução com Pyspark: 2.26 segundos


In [25]:
df_pyspark_title_basics.count()

1848568

#### Os valores nulos estavam com a informação "\N" foi corrigida para nulo para ser possível visualizar as quantidades de nulos no DF

#### Pandas

In [26]:
pandas_start = time.time()

df_pandas_title_basics = df_pandas_title_basics.replace(r'\N', None)

pandas_end = time.time()
print(f'Tempo de execução com Pandas: {pandas_end - pandas_start:.2f} segundos')

Tempo de execução com Pandas: 104.33 segundos


#### Pyskpark

In [27]:
pyspark_start = time.time()

df_pyspark_title_basics = df_pyspark_title_basics.na.replace(r'\N', None)

pyspark_end = time.time()
print(f'Tempo de execução com Pyspark: {pyspark_end - pyspark_start:.2f} segundos')

Tempo de execução com Pyspark: 50.34 segundos


#### Ajustes nos tipos de dados
- Ano estréia e ano encerramento alterado para formato data
- Tempo de duração alterado para float
- is Adulto para booleano

#### Pandas

In [28]:
pandas_start = time.time()

df_pandas_title_basics[['startYear', 'endYear']] = df_pandas_title_basics[['startYear', 'endYear']].apply(pd.to_datetime, format='%Y')
df_pandas_title_basics['runtimeMinutes'] = df_pandas_title_basics['runtimeMinutes'].astype(float)
df_pandas_title_basics['isAdult'] = df_pandas_title_basics['isAdult'].astype(bool)

pandas_end = time.time()
print(f'Tempo de execução com Pandas: {pandas_end - pandas_start:.2f} segundos')

Tempo de execução com Pandas: 19.96 segundos


#### Pyspark

In [30]:
pyspark_start = time.time()

df_pyspark_title_basics = df_pyspark_title_basics.withColumn(
   'startYear',
    to_date(col('startYear'), 'yyyy')
).withColumn(
    'endYear',
    to_date(col('endYear'), 'yyyy')
).withColumn(
   'runtimeMinutes',
    col('runtimeMinutes').cast('float')
).withColumn(
   'isAdult',
    col('isAdult').cast('boolean')
)


pyspark_end = time.time()
print(f'Tempo de execução com Pyspark: {pyspark_end - pyspark_start:.2f} segundos')

Tempo de execução com Pyspark: 0.52 segundos


#### Verificando a contagem de dados para cada coluna, 'endYear' que está com 0, ou seja, todos dados nulos, será excluida da base

#### Pandas

In [31]:
df_pandas_title_basics.count()

tconst            1848568
titleType         1848568
primaryTitle      1848563
originalTitle     1848563
isAdult           1848568
startYear         1707289
endYear                 0
runtimeMinutes    1186208
genres            1759593
dtype: int64

In [32]:
df_pandas_title_basics.drop(columns=['endYear'], inplace=True)

#### Pyspark

In [33]:
df_pyspark_title_basics.count()

1848568

In [34]:
df_pyspark_title_basics = df_pyspark_title_basics.drop('endYear')

### Join/Merge para unir as avaliações e votos no df de filmes

#### Pandas

In [35]:
pandas_start = time.time()

df_pandas_filmes_avaliacoes = pd.merge(
    df_pandas_title_basics,
    df_pandas_title_ratings,
    on=['tconst'],
    how='left'
)

pandas_end = time.time()
print(f'Tempo de execução com Pandas: {pandas_end - pandas_start:.2f} segundos')

Tempo de execução com Pandas: 98.91 segundos


#### Pyspark

In [36]:
pyspark_start = time.time()

df_pyspark_filmes_avaliacoes  = df_pyspark_title_basics.join(
    df_pyspark_title_ratings,
    on='tconst',
    how='left'
)

pyspark_end = time.time()
print(f'Tempo de execução com Pyspark: {pyspark_end - pyspark_start:.2f} segundos')

Tempo de execução com Pyspark: 7.92 segundos


#### Possível notar que tem 1.319.489 dados nulos nas colunas de média de avaliações (averageRating) e para Numero de votos (numVotes)
#### Foi entendido que os filmes que estão sem avaliações ou votos, não receberam nenhuma avaliação ou votos na plataforma

In [37]:
df_pandas_filmes_avaliacoes.isnull().sum()

tconst                  0
titleType               0
primaryTitle            5
originalTitle           5
isAdult                 0
startYear          141279
runtimeMinutes     662360
genres              88975
averageRating     1319489
numVotes          1319489
dtype: int64

#### Dropar os dados nulos para não afetar a análise

#### Pandas

In [38]:
pandas_start = time.time()

df_pandas_filmes_avaliacoes = df_pandas_filmes_avaliacoes.dropna(subset=['averageRating','startYear','runtimeMinutes','genres'])

pandas_end = time.time()
print(f'Tempo de execução com Pandas: {pandas_end - pandas_start:.2f} segundos')

Tempo de execução com Pandas: 53.13 segundos


In [39]:
df_pandas_filmes_avaliacoes.isnull().sum()

tconst            0
titleType         0
primaryTitle      0
originalTitle     0
isAdult           0
startYear         0
runtimeMinutes    0
genres            0
averageRating     0
numVotes          0
dtype: int64

In [40]:
df_pandas_filmes_avaliacoes.shape

(463632, 10)

#### Pyspark

In [41]:
df_pyspark_filmes_avaliacoes = df_pyspark_filmes_avaliacoes.dropna()

In [42]:
df_pyspark_filmes_avaliacoes.count()

463632

#### Criado um novo DataFrame pois os generos estão juntos. Fazendo a separação para que as análises por generos seja mais assertiva

#### Pandas 

- Fazendo uma cópia do df_pandas_filmes_avaliacoes que possui as informações incluindo as avaliações

In [43]:
df_pandas_contagem_genero = df_pandas_filmes_avaliacoes.copy()

- Duplicando a coluna de generos para uma delas ser subdividida pelos filmes que possui mais de um genero

In [44]:
pandas_start = time.time()

df_pandas_contagem_genero = df_pandas_contagem_genero.assign(genres_sub=df_pandas_contagem_genero['genres'])

pandas_end = time.time()
print(f'Tempo de execução com Pandas: {pandas_end - pandas_start:.2f} segundos')

Tempo de execução com Pandas: 71.15 segundos


- Utilizado o método .explode no qual se passsa uma coluna como parametro para expandir  mantendo as demais colunas do df inalteradas
- Utilizado para manter a informação dos generos de forma separadas mas permanecer o titulo e demais informações sobre o filme

In [45]:
pandas_start = time.time()

df_pandas_contagem_genero = df_pandas_contagem_genero.assign(genres=df_pandas_contagem_genero['genres'].str.split(',')).explode('genres').reset_index(drop=True)

pandas_end = time.time()
print(f'Tempo de execução com Pandas: {pandas_end - pandas_start:.2f} segundos')

Tempo de execução com Pandas: 92.96 segundos


In [46]:
df_pandas_contagem_genero.head(5)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes,genres_sub
0,tt0000001,short,Carmencita,Carmencita,False,1894-01-01,1.0,Documentary,5.7,2063.0,"Documentary,Short"
1,tt0000001,short,Carmencita,Carmencita,False,1894-01-01,1.0,Short,5.7,2063.0,"Documentary,Short"
2,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,False,1892-01-01,5.0,Animation,5.6,279.0,"Animation,Short"
3,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,False,1892-01-01,5.0,Short,5.6,279.0,"Animation,Short"
4,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,False,1892-01-01,5.0,Animation,6.5,2030.0,"Animation,Comedy,Romance"


#### Pyspark

- Fazendo uma cópia do df_pyspark_filmes_avaliacoes que possui as informações incluindo as avaliações

In [47]:
df_pyspark_contagem_genero = df_pyspark_filmes_avaliacoes

- Duplicando a coluna de generos para uma delas ser subdividida pelos filmes que possui mais de um genero

In [49]:
pyspark_start = time.time()

df_pyspark_contagem_genero = df_pyspark_contagem_genero.withColumn(
    'genres_sub',
    col('genres')
)

pyspark_end = time.time()
print(f'Tempo de execução com Pyspark: {pyspark_end - pyspark_start:.2f} segundos')

Tempo de execução com Pyspark: 0.09 segundos


- Assim como no pandas, utilizado o explode no qual se passsa uma coluna como parametro para expandir  mantendo as demais colunas do df inalteradas
- Utilizado para manter a informação dos generos de forma separadas mas permanecer o titulo e demais informações sobre o filme

In [50]:
pyspark_start = time.time()

df_pyspark_contagem_genero = df_pyspark_contagem_genero.withColumn(
    'genres', split(col('genres'), ',')
).withColumn('genres', explode(col('genres'))
)

pyspark_end = time.time()
print(f'Tempo de execução com Pyspark: {pyspark_end - pyspark_start:.2f} segundos')

Tempo de execução com Pyspark: 4.46 segundos


In [51]:
df_pyspark_contagem_genero.show(5)

+---------+---------+--------------------+--------------------+-------+----------+--------------+-----------+-------------+--------+-----------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult| startYear|runtimeMinutes|     genres|averageRating|numVotes|       genres_sub|
+---------+---------+--------------------+--------------------+-------+----------+--------------+-----------+-------------+--------+-----------------+
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|  false|1892-01-01|           5.0|  Animation|          5.6|     279|  Animation,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|  false|1892-01-01|           5.0|      Short|          5.6|     279|  Animation,Short|
|tt0000004|    short|         Un bon bock|         Un bon bock|  false|1892-01-01|          12.0|  Animation|          5.4|     180|  Animation,Short|
|tt0000004|    short|         Un bon bock|         Un bon bock|  false|1892-01-01|          12

### 11. Quais são os gêneros mais populares em cada década?

#### Pandas

In [52]:
df_pandas_populares_decada = df_pandas_contagem_genero.copy()

- Criando uma coluna decada e aplicando a função para que converta os anos em decadas

- Como busca apenas as decadas por exemplo, 1980, 1990, etc.
- Aplicado na coluna do ano de estréia, divisão inteira por 10 e multiplicado por 10 - Ex. 1878 / 10 = 187,80 (parte inteira 187 * 10 = 1870)

In [53]:
pandas_start = time.time()

df_pandas_populares_decada['Decada'] = df_pandas_populares_decada['startYear'].apply(lambda x: (x.year // 10) * 10)

pandas_end = time.time()
print(f'Tempo de execução com Pandas: {pandas_end - pandas_start:.2f} segundos')

Tempo de execução com Pandas: 73.25 segundos


- Agrupando os generos por cada década, fazendo uma contagem para verificar quais mais apareciam em cada uma das décadas

In [54]:
pandas_start = time.time()

df_pandas_generos_decada = df_pandas_populares_decada.groupby('genres')['Decada'].value_counts().reset_index()
df_pandas_generos_decada = df_pandas_generos_decada.rename(columns={'genres':'Genero', 'count': 'Contagem_Filmes'})
df_pandas_generos_decada

pandas_end = time.time()
print(f'Tempo de execução com Pandas: {pandas_end - pandas_start:.2f} segundos')

Tempo de execução com Pandas: 1.97 segundos


In [55]:
df_pandas_generos_decada

,Genero,Decada,Contagem_Filmes
0,Action,2010,12192
1,Action,2000,5683
2,Action,1990,4050
3,Action,2020,3966
4,Action,1980,3510
...,...,...,...
355,Western,2020,190
356,Western,1990,169
357,Western,1980,143
358,Western,1900,18


#### Pyspark

In [56]:
df_pyspark_populares_decada = df_pyspark_contagem_genero

- Criando uma coluna decada e aplicando a função para que converta os anos em decadas

In [57]:
pyspark_start = time.time()

df_pyspark_populares_decada = df_pyspark_populares_decada.withColumn(
    'Decada',
    floor(year(col('startYear')) / 10) * 10
)

pyspark_end = time.time()
print(f'Tempo de execução com Pyspark: {pyspark_end - pyspark_start:.2f} segundos')

Tempo de execução com Pyspark: 19.54 segundos


In [58]:
df_pyspark_populares_decada.show(5, False, True)

-RECORD 0------------------------------------------------
 tconst         | tt0000002                              
 titleType      | short                                  
 primaryTitle   | Le clown et ses chiens                 
 originalTitle  | Le clown et ses chiens                 
 isAdult        | false                                  
 startYear      | 1892-01-01                             
 runtimeMinutes | 5.0                                    
 genres         | Animation                              
 averageRating  | 5.6                                    
 numVotes       | 279                                    
 genres_sub     | Animation,Short                        
 Decada         | 1890                                   
-RECORD 1------------------------------------------------
 tconst         | tt0000002                              
 titleType      | short                                  
 primaryTitle   | Le clown et ses chiens                 
 originalTitle

- Agrupando os generos por cada década, fazendo uma contagem para verificar quais mais apareciam em cada uma das décadas

In [59]:
pyspark_start = time.time()

df_pyspark_generos_decada = df_pyspark_populares_decada.groupby('genres','Decada').count()
df_pyspark_generos_decada = df_pyspark_generos_decada.withColumnRenamed(
    'genres', 'Genero'
).withColumnRenamed(
    'count', 'Contagem_Filmes'
)

pyspark_end = time.time()
print(f'Tempo de execução com Pyspark: {pyspark_end - pyspark_start:.2f} segundos')

Tempo de execução com Pyspark: 1.39 segundos


In [60]:
df_pyspark_generos_decada.show()

+-----------+------+---------------+
|     Genero|Decada|Contagem_Filmes|
+-----------+------+---------------+
|Documentary|  1930|            637|
|      Short|  1970|           3986|
|        War|  1970|            680|
| Reality-TV|  2010|             69|
|    Western|  2000|            256|
|  Game-Show|  2000|              2|
|      Crime|  1940|           1185|
|     Comedy|  1970|           6826|
|     Comedy|  2010|          35847|
|     Horror|  1950|            322|
|  Adventure|  2000|           2790|
|Documentary|  1960|           1974|
|     Sci-Fi|  1990|           1059|
|Documentary|  1990|           4333|
|Documentary|  1880|             18|
|     Family|  1920|            216|
|       News|  1930|             16|
|  Biography|  1960|            270|
|     Sci-Fi|  2000|           2082|
|      Sport|  1980|            250|
+-----------+------+---------------+
only showing top 20 rows



- Criar um ranking dos melhores em cada década

#### Pandas

In [61]:
pandas_start = time.time()

df_pandas_ranking = df_pandas_generos_decada.sort_values(by=['Decada','Contagem_Filmes'],ascending=False).reset_index(drop=True)

pandas_end = time.time()
print(f'Tempo de execução com Pandas: {pandas_end - pandas_start:.2f} segundos')

Tempo de execução com Pandas: 4.27 segundos


In [62]:
df_pandas_ranking['Decada'].unique()

array([2020, 2010, 2000, 1990, 1980, 1970, 1960, 1950, 1940, 1930, 1920,
       1910, 1900, 1890, 1880, 1870], dtype=int64)

In [63]:
pandas_start = time.time()

df_pandas_ranking['Classificacao'] = df_pandas_ranking.groupby('Decada').cumcount() + 1

pandas_end = time.time()
print(f'Tempo de execução com Pandas: {pandas_end - pandas_start:.2f} segundos')

Tempo de execução com Pandas: 2.12 segundos


In [64]:
df_pandas_ranking.head()

,Genero,Decada,Contagem_Filmes,Classificacao
0,Drama,2020,20616,1
1,Short,2020,13725,2
2,Documentary,2020,11539,3
3,Comedy,2020,10563,4
4,Horror,2020,6029,5


#### Pyspark

In [65]:
pyspark_start = time.time()

df_pyspark_ranking = df_pyspark_generos_decada.orderBy(
    ['Decada', 'Contagem_Filmes'],
    ascending=[True, False]
)

pyspark_end = time.time()
print(f'Tempo de execução com Pyspark: {pyspark_end - pyspark_start:.2f} segundos')

Tempo de execução com Pyspark: 20.42 segundos


In [66]:
pyspark_start = time.time()

classificacao = Window.partitionBy('Decada').orderBy(col('Contagem_filmes').desc())
df_pyspark_ranking = df_pyspark_ranking.withColumn(
    'Classificacao', row_number().over(classificacao))

pyspark_end = time.time()
print(f'Tempo de execução com Pyspark: {pyspark_end - pyspark_start:.2f} segundos')

Tempo de execução com Pyspark: 12.72 segundos


In [70]:
df_pyspark_ranking.show(5)

+-----------+------+---------------+-------------+
|     Genero|Decada|Contagem_Filmes|Classificacao|
+-----------+------+---------------+-------------+
|      Short|  1870|             30|            1|
|  Animation|  1870|             28|            2|
|Documentary|  1870|              2|            3|
|      Sport|  1870|              1|            4|
|      Short|  1880|             51|            1|
+-----------+------+---------------+-------------+
only showing top 5 rows



### Ordenação O Pandas tem maior performance

#### Pandas

- Top 1 para cada decada

In [67]:
df_pandas_ranking[df_pandas_ranking['Classificacao'] ==1]

,Genero,Decada,Contagem_Filmes,Classificacao
0,Drama,2020,20616,1
27,Drama,2010,66641,1
54,Drama,2000,32798,1
81,Drama,1990,15472,1
107,Drama,1980,13478,1
133,Drama,1970,12250,1
159,Drama,1960,9693,1
185,Drama,1950,7011,1
210,Drama,1940,4705,1
235,Comedy,1930,5166,1


#### Pyspark

- Top 1 para cada decada

In [68]:
df_pyspark_ranking_top1= df_pyspark_ranking.filter(col('Classificacao') == 1).orderBy(col('Decada').desc())
df_pyspark_ranking_top1.show()

+------+------+---------------+-------------+
|Genero|Decada|Contagem_Filmes|Classificacao|
+------+------+---------------+-------------+
| Drama|  2020|          20616|            1|
| Drama|  2010|          66641|            1|
| Drama|  2000|          32798|            1|
| Drama|  1990|          15472|            1|
| Drama|  1980|          13478|            1|
| Drama|  1970|          12250|            1|
| Drama|  1960|           9693|            1|
| Drama|  1950|           7011|            1|
| Drama|  1940|           4705|            1|
|Comedy|  1930|           5166|            1|
| Drama|  1920|           2207|            1|
| Short|  1910|           2334|            1|
| Short|  1900|           1878|            1|
| Short|  1890|            908|            1|
| Short|  1880|             51|            1|
| Short|  1870|             30|            1|
+------+------+---------------+-------------+



### Conclusão:
### Pandas ou Pyspark?

```` word

Não existe uma regra exata. 

Vai depender de cada situação e tamanho das bases de dados.
Como pode ser observado nas comparações acima, o Pyspark em grande parte foi mais rápido, exceto se tratando de ordenação dos dados para visualização em que o Pandas se sobressai. 

De qualquer forma, tem outros fatores que precisam ser levados em consideração dependendo da situação. O Pandas possui uma visualização mais "amigável" do que o Pyspark e trabalhando com volumes menores de dados pode não ter impactos relevantes de performance se comparado ao Pyspark.

Uma possibilidade é a flexibilidade de poder trabalhar com ambas em uma mesma análise.